### Intraday Trading Strategy

Buying Condition: If the return after two hours breaks x threshold, buy the asset 

Selling Condition: The asset drops until a certain xn threshold OR asset climbs to xn threshold 

In [34]:
import yfinance as yf 
import pandas as pd 
import datetime as dt
import numpy as np

In [2]:
asset = 'TSLA'
intraday = yf.download(asset, start='2022-02-07', end='2022-02-10', interval='1m')

[*********************100%***********************]  1 of 1 completed


In [3]:
intraday

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-02-07 09:30:00-05:00,923.789978,924.579895,918.340027,922.809998,922.809998,652911
2022-02-07 09:31:00-05:00,922.500000,927.000000,921.500122,924.659973,924.659973,128771
2022-02-07 09:32:00-05:00,925.510010,929.066528,924.840027,927.010498,927.010498,162178
2022-02-07 09:33:00-05:00,927.190002,928.200012,922.510071,922.566772,922.566772,129453
2022-02-07 09:34:00-05:00,922.510010,922.979980,919.330078,920.469971,920.469971,121114
...,...,...,...,...,...,...
2022-02-09 15:56:00-05:00,931.599976,932.000000,931.330017,931.460022,931.460022,57791
2022-02-09 15:57:00-05:00,931.469971,931.659973,930.919983,931.479980,931.479980,68651
2022-02-09 15:58:00-05:00,931.525024,931.630005,930.150085,930.500000,930.500000,99340


In [48]:
def IntradayTrend(df,entry,exit):
    ret_120min = df.iloc[120].Open / df.iloc[0].Open -1
    ticketret = df.Open.pct_change()
    exitprice = 0
    if ret_120min > entry:
        buyprice = df.iloc[121].Open
        buytime = df.iloc[121].name
        cumulated = (ticketret.loc[buytime:] + 1).cumprod() - 1 
        exittime = cumulated[(cumulated < -exit) | (cumulated > exit)].first_valid_index()
        if exittime == None:
            exitprice = df.iloc[-1].Open
        else:
            exitprice = df.loc[exittime].shift(-1).Open
        profit = (exitprice - buyprice)
        profitrel = (profit / buyprice)
        return profitrel
    else:
        return None

In [17]:
datesframe = yf.download(asset,start='2022-02-07', end='2022-02-28')

[*********************100%***********************]  1 of 1 completed


In [18]:
datesframe.index

DatetimeIndex(['2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10',
               '2022-02-11', '2022-02-14', '2022-02-15', '2022-02-16',
               '2022-02-17', '2022-02-18', '2022-02-22', '2022-02-23',
               '2022-02-24', '2022-02-25'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [19]:
frames = []
for i in datesframe.index:
    frames.append(yf.download(asset,start=i,end=i+dt.timedelta(days=1),interval='1m'))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [49]:
returns = []

for i in frames:
    returns.append(IntradayTrend(i,0.01,0.01))

In [50]:
returns

[None,
 -0.041618518629318285,
 None,
 -0.006458834082976484,
 None,
 -0.009322672721751957,
 -0.03993494307185804,
 None,
 None,
 None,
 None,
 None,
 -0.01130928432347859,
 None]